In [1]:
import json
from collections import defaultdict
from pymongo import MongoClient, ASCENDING
from time import time
from pprint import pprint as pp

In [2]:
def insert_json(col):
    with open(data_paths[col], encoding='utf-8') as file:
        data = file.read()
        data = json.loads(data)
        ids = collections[col].insert_many(data).inserted_ids
    return ids

In [3]:
client = MongoClient('localhost', 27017)
database = client['db_exam']
collections = {'promo_codes':database['promo_codes'],'orders':database['orders']}
data_paths = {'promo_codes':'mongo-db/data/promo_codes.json','orders':'mongo-db/data/orders.json'}

In [4]:
# for col in data_paths:
#     insert_json(col)

In [6]:
def get_orders(user_id=None):
    d = {'user_id':user_id} if user_id else dict()
    
    data = collections['orders'].find(d)
    
    res = []
    for x in data:
        total = 0
        for item in x['items']:
            total += item['amount']*item['price']
        x['price'] = total
        res.append(x)
    return res
        
pp(get_orders(7487)[0])

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [21]:
def use_promo_code(code):
    res = collections['promo_codes'].find_one({'id':code})
    if res['expires'] < time():
        return 'Promo Code is expired'
    modified = collections['promo_codes'].update_one({'id':code},{ "$set": {'used':'true'}}).modified_count
    if modified == 0:
        return 'Promo Code can only be used one time'
    return 'Promo Code used'

use_promo_code('gjsormci')

'Promo Code is expired'